# 6-1 셀레니움 설치 및 드라이브 불러오기

In [2]:
# ! pip install selenium

In [3]:
# ! pip install webdriver-manager

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

###################
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
####################

import warnings
warnings.filterwarnings("ignore")

In [50]:
# driver를 설치하여 열기
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\han19\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


In [53]:
# 설치된 driver를 열기
s = Service("driver/chromedriver.exe")
driver = webdriver.Chrome(service=s)

# 6-2 셀레니움 사용법 익히기

In [10]:
# Window 창을 최대화 
driver.maximize_window()

In [54]:
# Url 주소를 가져와 이동
url = "https://www.google.co.kr/imghp?hl=ko"
driver.get(url)

## 키보드 이벤트

In [40]:
driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input').send_keys('서울')

In [16]:
driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input').send_keys('지하철')

In [17]:
driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input').send_keys('노선도')

## 마우스 이벤트

In [41]:
driver.find_element_by_xpath('//*[@id="sbtc"]/button').click()

## 스크롤 이벤트

In [42]:
# 현재 스크롤 위치
driver.execute_script("return window.pageYOffset")

0

In [43]:
# 가장 밑에 스크롤 위치
driver.execute_script("return document.body.scrollHeight")

3257

In [27]:
# 0에서 3872 이동
driver.execute_script("window.scrollTo(0,3872);")

In [28]:
# 현재 스크롤 위치
driver.execute_script("return window.pageYOffset")

3872

# 6-3 구글 이미지 스크롤 내리는 함수 만들기

## For문을 활용한 스크롤 내리기

In [45]:
import time 
for i in range(6):
    # 현재 스크롤 위치
    initial_page = driver.execute_script("return window.pageYOffset")
    # 맨 마지막 스크롤 위치 
    last_page = driver.execute_script("return document.body.scrollHeight")
    # 스크롤 이동 
    driver.execute_script("window.scrollTo("+str(initial_page)+","+str(last_page)+");")
    time.sleep(2.0) # 2.0 휴식 -> 로딩타임을 위해서

In [44]:
# 결과 더보기 클릭
driver.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input').click()

In [82]:
def scroll_down():
    for i in range(6):
        # 현재 스크롤 위치
        initial_page = driver.execute_script("return window.pageYOffset")
        # 맨 마지막 스크롤 위치 
        last_page = driver.execute_script("return document.body.scrollHeight")
        # 스크롤 이동 
        driver.execute_script("window.scrollTo("+str(initial_page)+","+str(last_page)+");")
        time.sleep(1.0) # 2.0 휴식 -> 로딩타임을 위해서

# 6-4 이미지 소스 찾아서 저장하기

In [32]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [38]:
img = soup.find("div","islrc").find_all('img')[0]['src']

In [46]:
import urllib.request
urllib.request.urlretrieve(img,'seoul01.jpg')

('seoul01.jpg', <email.message.Message at 0x17e9ce9faf0>)

In [56]:
import os 
os.makedirs('강아지')

# 6-5 하나의 함수로 묶어서 정리하기

In [86]:
def image_downloader():
    # Url 주소를 가져와 이동
    url = "https://www.google.co.kr/imghp?hl=ko"
    driver.get(url)
    keyword = input("다운받을 이미지 이름을 입력해주세요. ")
    driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input').send_keys(keyword)
    driver.find_element_by_xpath('//*[@id="sbtc"]/button').click()
    scroll_down()
    # 결과 더보기
    driver.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input').click()
    scroll_down()
    soup = BeautifulSoup(driver.page_source, "html.parser")
    img_list = []
    for i in range(len(soup.find("div","islrc").find_all('img'))):
        try:
            img_list.append(soup.find("div","islrc").find_all('img')[i]['src'])
        except:
            img_list.append(soup.find("div","islrc").find_all('img')[i]['src'])
    print("현재 이미지가 {}개 있습니다.".format(len(img_list)))
    num = int(input("다운받을 이미지 개수를 입력해주세요. "))
    os.makedirs(keyword)
    for i in range(num):
        url = img_list[i]
        urllib.request.urlretrieve(url,keyword+'/'+keyword+str(i)+'.jpg')
    print("다운이 완료되었습니다.")